In [23]:
import os
import sklearn
import numpy as np
import matplotlib.pyplot as plt

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

from scipy import ndimage

from keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.externals import joblib
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split


In [18]:
def load_imgs():
    img_datagen = ImageDataGenerator(rescale=1/255,  
                       )
    
    img_generator = img_datagen.flow_from_directory(
        './data',
        target_size=(110, 110),
        batch_size=10000,
        class_mode='categorical'
    )

    x, y = img_generator.next()
    
    return x, y


In [19]:
x, y = load_imgs()

Found 2966 images belonging to 286 classes.


In [48]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [49]:
old_shape = x_train.shape
x_train_res = x_train.reshape(old_shape[0], old_shape[1] * old_shape[2]*old_shape[3])

old_shape = x_test.shape
x_test_res = x_test.reshape(old_shape[0], old_shape[1] * old_shape[2]*old_shape[3])

In [50]:
pca = PCA(n_components=286)

x_train_reduced = pca.fit_transform(x_train_res)
x_test_reduced = pca.fit_transform(x_test_res)

In [51]:
y_train = np.array([np.where(r==1)[0][0] for r in y_train]) 
y_test  =np.array([np.where(r==1)[0][0] for r in y_test])

In [52]:
rf = RandomForestClassifier(n_estimators=100)
rf.fit(x_train_reduced, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [53]:
y_pred = rf.predict(x_test_reduced)

print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.00      0.00      0.00         3
           2       0.00      0.00      0.00         2
           3       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         5
          10       0.00      0.00      0.00         1
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         3
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         1
          15       0.00      0.00      0.00         4
          16       0.00      0.00      0.00         0
          20       0.00      0.00      0.00         2
          21       0.00    

/home/igorbispo/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/igorbispo/.local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
